# Purpose

This notebook looks at Natural Stat Trick data and parses/cleans it up for use in future machine learning.


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import re

In [ ]:
# start with the first year
df_2008 = pd.read_csv('nst_regular_season/2008.csv')

# compare columns with the last year
df_2021 = pd.read_csv('nst_regular_season/2021.csv')

print(df_2008.dtypes)
print(df_2021.dtypes)

# Both of these have 59 columns, so data is consistent across years

Game           object
Team           object
Unnamed: 2     object
TOI           float64
CF              int64
CA              int64
CF%           float64
FF              int64
FA              int64
FF%           float64
SF              int64
SA              int64
SF%           float64
GF              int64
GA              int64
GF%            object
xGF           float64
xGA           float64
xGF%           object
SCF             int64
SCA             int64
SCF%           object
HDCF            int64
HDCA            int64
HDCF%          object
HDSF            int64
HDSA            int64
HDSF%          object
HDGF            int64
HDGA            int64
HDGF%          object
HDSH%          object
HDSV%          object
MDCF            int64
MDCA            int64
MDCF%          object
MDSF            int64
MDSA            int64
MDSF%          object
MDGF            int64
MDGA            int64
MDGF%          object
MDSH%          object
MDSV%          object
LDCF            int64
LDCA      

In [ ]:
# Combine all CSVs into a single DF
combined_df = pd.read_csv('nst_regular_season/2008.csv')
for i in range(2009, 2022):
    temp_df = pd.read_csv('nst_regular_season/' + str(i) + '.csv')
    # Ignore Index is important for a later step, because we need unique indexes to handle row swaps
    combined_df = combined_df.append(temp_df, ignore_index=True)

combined_df

,Game,Team,Unnamed: 2,TOI,CF,CA,CF%,FF,FA,FF%,...,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Attendance
0,"2007-09-29 - Ducks 1, Kings 4",Anaheim Ducks,Limited ReportFull Report,34.700000,28,11,71.79,18,9,66.67,...,100.0,0,0,-,0.00,-,0.00,100.00,1.000,17300
1,"2007-09-29 - Ducks 1, Kings 4",Los Angeles Kings,Limited ReportFull Report,34.700000,11,28,28.21,9,18,33.33,...,0.0,0,0,-,-,100.00,0.00,100.00,1.000,17300
2,"2007-09-30 - Kings 1, Ducks 4",Anaheim Ducks,Limited ReportFull Report,48.200000,33,30,52.38,26,26,50.00,...,33.33,0,0,-,0.00,100.00,10.53,95.00,1.055,17300
3,"2007-09-30 - Kings 1, Ducks 4",Los Angeles Kings,Limited ReportFull Report,48.200000,30,33,47.62,26,26,50.00,...,66.67,0,0,-,0.00,100.00,5.00,89.47,0.945,17300
4,"2007-10-03 - Canadiens 3, Hurricanes 2",Carolina Hurricanes,Limited ReportFull Report,37.166667,45,51,46.88,32,32,50.00,...,62.5,1,0,100.00,10.00,100.00,4.55,100.00,1.045,18680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32559,"2021-05-16 - Flames 6, Canucks 5",Vancouver Canucks,Limited ReportFull Report,40.966667,32,50,39.02,25,34,42.37,...,33.33,1,1,50.00,14.29,92.86,8.70,78.26,0.870,0
32560,"2021-05-18 - Flames 2, Canucks 4",Calgary Flames,Limited ReportFull Report,47.416667,43,37,53.75,35,30,53.85,...,55.56,0,0,-,0.0,100.0,3.33,86.36,0.897,0
32561,"2021-05-18 - Flames 2, Canucks 4",Vancouver Canucks,Limited ReportFull Report,47.416667,37,43,46.25,30,35,46.15,...,44.44,0,0,-,0.0,100.0,13.64,96.67,1.103,0
32562,"2021-05-19 - Canucks 2, Flames 6",Calgary Flames,Limited ReportFull Report,44.483333,43,35,55.13,37,29,56.06,...,42.86,0,0,-,0.0,100.0,24.00,94.44,1.184,0


In [ ]:
# Clean up columns we don't need

# Unnamed: 2 -> links to the NST reports
# TOI -> irrelevant
# CF/CA/CF% -> fenwick is more popular than corsi nowadays
# HDGF/HDGA/HDGF%/HDGF%/HDSH%/HDSV%/M*,L* -> goals are subject to entropy, so it's better to look at shots/chances overall
# SH% and SV% -> captured via PDO
# Attendance -> I can't see how past attendance would be a future predictor of performance

combined_df.drop(columns=['Unnamed: 2', 'TOI', 'CF', 'CA', 'CF%', 'SH%', 'SV%', 'Attendance',\
    'HDGF', 'HDGA', 'HDGF%', 'HDSH%', 'HDSV%', 'MDGF', 'MDGA', 'MDGF%', 'MDSH%', 'MDSV%',\
    'LDGF', 'LDGA', 'LDGF%', 'LDSH%', 'LDSV%'],\
    inplace = True)

# Drop columns that are %, as those can be inferred. And drop columns that are duplicated across home/visitor rows, eg GF for home == GA for visitor
combined_df.drop(columns=['FA', 'FF%', 'SA', 'SF%', 'GA', 'GF%', 'xGA', 'xGF%', 'SCA', 'SCF%', 'HDCA', 'HDCF%',\
    'HDSA', 'HDSF%', 'MDCA', 'MDCF%', 'MDSA', 'MDSF%', 'LDCA', 'LDCF%', 'LDSA', 'LDSF%'],\
    inplace = True)


combined_df

,Game,Team,FF,SF,GF,xGF,SCF,HDCF,HDSF,MDCF,MDSF,LDCF,LDSF,PDO
0,"2007-09-29 - Ducks 1, Kings 4",Anaheim Ducks,18,14,0,1.28,15,6,4,9,5,12,5,1.000
1,"2007-09-29 - Ducks 1, Kings 4",Los Angeles Kings,9,6,0,0.34,7,0,0,7,5,2,0,1.000
2,"2007-09-30 - Kings 1, Ducks 4",Anaheim Ducks,26,19,2,1.53,19,9,8,10,5,10,5,1.055
3,"2007-09-30 - Kings 1, Ducks 4",Los Angeles Kings,26,20,1,1.20,16,3,3,13,7,13,10,0.945
4,"2007-10-03 - Canadiens 3, Hurricanes 2",Carolina Hurricanes,32,22,1,2.15,26,12,7,14,5,15,10,1.045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32559,"2021-05-16 - Flames 6, Canucks 5",Vancouver Canucks,25,23,2,1.33,17,3,3,14,11,11,7,0.870
32560,"2021-05-18 - Flames 2, Canucks 4",Calgary Flames,35,30,1,2.36,25,10,9,15,11,15,10,0.897
32561,"2021-05-18 - Flames 2, Canucks 4",Vancouver Canucks,30,22,3,1.32,17,6,5,11,9,17,8,1.103
32562,"2021-05-19 - Canucks 2, Flames 6",Calgary Flames,37,25,6,3.15,30,19,13,11,6,13,6,1.184


In [ ]:
combined_df2 = combined_df

# Extract the date into a separate column
# The date will always be in the format YYYY-MM-DD. To make it easy, we can just take the first (4+1+2+1+2) chars of the string

# Example: "2008-04-06 - Wild 3, Avalanche 4"
def format_date(game_string):
    datestr = game_string[0:10]
    return dt.datetime.strptime(datestr, '%Y-%m-%d')

combined_df2['Date'] = combined_df2.apply(lambda row: format_date(str(row.Game)), axis=1)
combined_df2



,Game,Team,FF,SF,GF,xGF,SCF,HDCF,HDSF,MDCF,MDSF,LDCF,LDSF,PDO,Date
0,"2007-09-29 - Ducks 1, Kings 4",Anaheim Ducks,18,14,0,1.28,15,6,4,9,5,12,5,1.000,2007-09-29
1,"2007-09-29 - Ducks 1, Kings 4",Los Angeles Kings,9,6,0,0.34,7,0,0,7,5,2,0,1.000,2007-09-29
2,"2007-09-30 - Kings 1, Ducks 4",Anaheim Ducks,26,19,2,1.53,19,9,8,10,5,10,5,1.055,2007-09-30
3,"2007-09-30 - Kings 1, Ducks 4",Los Angeles Kings,26,20,1,1.20,16,3,3,13,7,13,10,0.945,2007-09-30
4,"2007-10-03 - Canadiens 3, Hurricanes 2",Carolina Hurricanes,32,22,1,2.15,26,12,7,14,5,15,10,1.045,2007-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32559,"2021-05-16 - Flames 6, Canucks 5",Vancouver Canucks,25,23,2,1.33,17,3,3,14,11,11,7,0.870,2021-05-16
32560,"2021-05-18 - Flames 2, Canucks 4",Calgary Flames,35,30,1,2.36,25,10,9,15,11,15,10,0.897,2021-05-18
32561,"2021-05-18 - Flames 2, Canucks 4",Vancouver Canucks,30,22,3,1.32,17,6,5,11,9,17,8,1.103,2021-05-18
32562,"2021-05-19 - Canucks 2, Flames 6",Calgary Flames,37,25,6,3.15,30,19,13,11,6,13,6,1.184,2021-05-19


In [ ]:
'''
# The only way we can determine if a team is home or not is based on the order of the Game row. First team is almost always away
game_away_regex = '[0-9]{4}-[0-9]{2}-[0-9]{2} - ([a-zA-Z]*) .*'

def is_away_row(row):
    away_team = re.search(game_away_regex, row.Game).group(1)
    return away_team in row.Team

combined_df2['away_row'] = combined_df2.apply(lambda row: is_away_row(row), axis=1)

'''

def is_away_row(row):
    return row.name % 2 == 0

combined_df2['away_row'] = combined_df2.apply(lambda row: is_away_row(row), axis=1)
combined_df2

,Game,Team,FF,SF,GF,xGF,SCF,HDCF,HDSF,MDCF,MDSF,LDCF,LDSF,PDO,Date,away_row
0,"2007-09-29 - Ducks 1, Kings 4",Anaheim Ducks,18,14,0,1.28,15,6,4,9,5,12,5,1.000,2007-09-29,True
1,"2007-09-29 - Ducks 1, Kings 4",Los Angeles Kings,9,6,0,0.34,7,0,0,7,5,2,0,1.000,2007-09-29,False
2,"2007-09-30 - Kings 1, Ducks 4",Anaheim Ducks,26,19,2,1.53,19,9,8,10,5,10,5,1.055,2007-09-30,True
3,"2007-09-30 - Kings 1, Ducks 4",Los Angeles Kings,26,20,1,1.20,16,3,3,13,7,13,10,0.945,2007-09-30,False
4,"2007-10-03 - Canadiens 3, Hurricanes 2",Carolina Hurricanes,32,22,1,2.15,26,12,7,14,5,15,10,1.045,2007-10-03,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32559,"2021-05-16 - Flames 6, Canucks 5",Vancouver Canucks,25,23,2,1.33,17,3,3,14,11,11,7,0.870,2021-05-16,False
32560,"2021-05-18 - Flames 2, Canucks 4",Calgary Flames,35,30,1,2.36,25,10,9,15,11,15,10,0.897,2021-05-18,True
32561,"2021-05-18 - Flames 2, Canucks 4",Vancouver Canucks,30,22,3,1.32,17,6,5,11,9,17,8,1.103,2021-05-18,False
32562,"2021-05-19 - Canucks 2, Flames 6",Calgary Flames,37,25,6,3.15,30,19,13,11,6,13,6,1.184,2021-05-19,True


In [ ]:
# Re-order the index, with the expectation that visitor is first, home is second (like with the other odds data)
# For each row, IF home is off (in an even place), move it down (+1)
# IF away is off (in an odd place), move it up (-1)

new_index = []
for index, row in combined_df2.iterrows():
    if not row.away_row and index % 2 == 0:
        new_index.append(index + 1)
    elif row.away_row and index % 2 == 1:
        new_index.append(index - 1)
    else:
        new_index.append(index)

reverse_df2 = combined_df2
reverse_df2['away_row'] = reverse_df2.apply(lambda row: not row.away_row, axis=1)

new_index2 = []
for index, row in reverse_df2.iterrows():
    if not row.away_row and index % 2 == 0:
        new_index2.append(index + 1)
    elif row.away_row and index % 2 == 1:
        new_index2.append(index - 1)
    else:
        new_index2.append(index)

# Now we're going to assume the opposite so we'll create a big DF. We'll end up dropping the wrong rows anyways


combined_df2 = combined_df2.reindex(index = new_index)
reverse_df2 = reverse_df2.reindex(index = new_index2)
combined_df2 = combined_df2.append(reverse_df2)
combined_df2

,Game,Team,FF,SF,GF,xGF,SCF,HDCF,HDSF,MDCF,MDSF,LDCF,LDSF,PDO,Date,away_row
0,"2007-09-29 - Ducks 1, Kings 4",Anaheim Ducks,18,14,0,1.28,15,6,4,9,5,12,5,1.000,2007-09-29,False
1,"2007-09-29 - Ducks 1, Kings 4",Los Angeles Kings,9,6,0,0.34,7,0,0,7,5,2,0,1.000,2007-09-29,True
2,"2007-09-30 - Kings 1, Ducks 4",Anaheim Ducks,26,19,2,1.53,19,9,8,10,5,10,5,1.055,2007-09-30,False
3,"2007-09-30 - Kings 1, Ducks 4",Los Angeles Kings,26,20,1,1.20,16,3,3,13,7,13,10,0.945,2007-09-30,True
4,"2007-10-03 - Canadiens 3, Hurricanes 2",Carolina Hurricanes,32,22,1,2.15,26,12,7,14,5,15,10,1.045,2007-10-03,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32558,"2021-05-16 - Flames 6, Canucks 5",Calgary Flames,34,23,5,1.73,19,7,5,12,4,26,14,1.130,2021-05-16,False
32561,"2021-05-18 - Flames 2, Canucks 4",Vancouver Canucks,30,22,3,1.32,17,6,5,11,9,17,8,1.103,2021-05-18,True
32560,"2021-05-18 - Flames 2, Canucks 4",Calgary Flames,35,30,1,2.36,25,10,9,15,11,15,10,0.897,2021-05-18,False
32563,"2021-05-19 - Canucks 2, Flames 6",Vancouver Canucks,29,18,1,1.65,18,6,4,12,5,15,8,0.816,2021-05-19,True


In [ ]:
# There are 33 teams in the DF (except 31). Get the unique list of teams to correct any overlap
combined_df2['Team'].unique()

array(['Anaheim Ducks', 'Los Angeles Kings', 'Carolina Hurricanes',
       'Montreal Canadiens', 'Detroit Red Wings', 'Ottawa Senators',
       'Toronto Maple Leafs', 'Colorado Avalanche', 'Dallas Stars',
       'Nashville Predators', 'Chicago Blackhawks', 'Minnesota Wild',
       'Phoenix Coyotes', 'St Louis Blues', 'New Jersey Devils',
       'Tampa Bay Lightning', 'Calgary Flames', 'Philadelphia Flyers',
       'Florida Panthers', 'New York Rangers', 'Edmonton Oilers',
       'San Jose Sharks', 'Boston Bruins', 'Atlanta Thrashers',
       'Washington Capitals', 'Columbus Blue Jackets', 'Buffalo Sabres',
       'New York Islanders', 'Pittsburgh Penguins', 'Vancouver Canucks',
       'Winnipeg Jets', 'Arizona Coyotes', 'Vegas Golden Knights'],
      dtype=object)

In [ ]:
# Phoenix Coyotes -> Arizona Coyotes
# Atlanta Thrashers -> Winnipeg Jets

# Seattle Kraken

name_changes = {'Phoenix Coyotes': 'Arizona Coyotes', 'Atlanta Thrashers': 'Winnipeg Jets'}

combined_df3 = combined_df2.replace({'Team': name_changes})

combined_df3['Team'].unique()

array(['Anaheim Ducks', 'Los Angeles Kings', 'Carolina Hurricanes',
       'Montreal Canadiens', 'Detroit Red Wings', 'Ottawa Senators',
       'Toronto Maple Leafs', 'Colorado Avalanche', 'Dallas Stars',
       'Nashville Predators', 'Chicago Blackhawks', 'Minnesota Wild',
       'Arizona Coyotes', 'St Louis Blues', 'New Jersey Devils',
       'Tampa Bay Lightning', 'Calgary Flames', 'Philadelphia Flyers',
       'Florida Panthers', 'New York Rangers', 'Edmonton Oilers',
       'San Jose Sharks', 'Boston Bruins', 'Winnipeg Jets',
       'Washington Capitals', 'Columbus Blue Jackets', 'Buffalo Sabres',
       'New York Islanders', 'Pittsburgh Penguins', 'Vancouver Canucks',
       'Vegas Golden Knights'], dtype=object)

In [ ]:
# Next, we need to clean up the names to match what we get from the other dataset

'''

array(['Pittsburgh', 'Philadelphia', 'Montreal', 'Toronto', 'Chicago',
       'TampaBay', 'Vancouver', 'Edmonton', 'St.Louis', 'Colorado',
       'Boston', 'NewJersey', 'Washington', 'Buffalo', 'NYIslanders',
       'NYRangers', 'Carolina', 'Detroit', 'Calgary', 'Winnipeg',
       'Columbus', 'Nashville', 'SanJose', 'Arizona', 'Minnesota',
       'LosAngeles', 'Anaheim', 'Vegas', 'Ottawa', 'Florida', 'Dallas',
       'NY Islanders', 'Tampa Bay'], dtype=object)

fixed_names = {'NY Islanders': 'NYIslanders', 'Tampa Bay': 'TampaBay'}
'''



'''
array(['LosAngeles', 'Detroit', 'Toronto', 'NYRangers', 'Nashville',
       'Minnesota', 'Columbus', 'Dallas', 'Vancouver', 'Ottawa',
       'Washington', 'Pittsburgh', 'Arizona', 'St.Louis', 'Buffalo',
       'Montreal', 'Colorado', 'SanJose', 'Chicago', 'Carolina',
       'Florida', 'NewJersey', 'Calgary', 'Boston', 'Winnipeg'],
      dtype=object)
'''

short_names = {'Anaheim Ducks': 'Anaheim', 'Los Angeles Kings': 'LosAngeles', 'Carolina Hurricanes': 'Carolina',\
    'Montreal Canadiens': 'Montreal', 'Detroit Red Wings': 'Detroit', 'Ottawa Senators': 'Ottawa',\
    'Toronto Maple Leafs': 'Toronto', 'Colorado Avalanche': 'Colorado', 'Dallas Stars': 'Dallas',\
    'Nashville Predators': 'Nashville', 'Chicago Blackhawks': 'Chicago', 'Minnesota Wild': 'Minnesota',\
    'Arizona Coyotes': 'Arizona', 'St Louis Blues': 'St.Louis', 'New Jersey Devils': 'NewJersey',\
    'Tampa Bay Lightning': 'TampaBay', 'Calgary Flames': 'Calgary', 'Philadelphia Flyers': 'Philadelphia',\
    'Florida Panthers': 'Florida', 'New York Rangers': 'NYRangers', 'Edmonton Oilers': 'Edmonton',\
    'San Jose Sharks': 'SanJose', 'Boston Bruins': 'Boston', 'Winnipeg Jets': 'Winnipeg',\
    'Washington Capitals': 'Washington', 'Columbus Blue Jackets': 'Columbus', 'Buffalo Sabres': 'Buffalo',\
    'New York Islanders': 'NYIslanders', 'Pittsburgh Penguins': 'Pittsburgh', 'Vancouver Canucks': 'Vancouver',\
    'Vegas Golden Knights' : 'Vegas'}

combined_df3 = combined_df3.replace({'Team': short_names})

combined_df3



,Game,Team,FF,SF,GF,xGF,SCF,HDCF,HDSF,MDCF,MDSF,LDCF,LDSF,PDO,Date,away_row
0,"2007-09-29 - Ducks 1, Kings 4",Anaheim,18,14,0,1.28,15,6,4,9,5,12,5,1.000,2007-09-29,False
1,"2007-09-29 - Ducks 1, Kings 4",LosAngeles,9,6,0,0.34,7,0,0,7,5,2,0,1.000,2007-09-29,True
2,"2007-09-30 - Kings 1, Ducks 4",Anaheim,26,19,2,1.53,19,9,8,10,5,10,5,1.055,2007-09-30,False
3,"2007-09-30 - Kings 1, Ducks 4",LosAngeles,26,20,1,1.20,16,3,3,13,7,13,10,0.945,2007-09-30,True
4,"2007-10-03 - Canadiens 3, Hurricanes 2",Carolina,32,22,1,2.15,26,12,7,14,5,15,10,1.045,2007-10-03,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32558,"2021-05-16 - Flames 6, Canucks 5",Calgary,34,23,5,1.73,19,7,5,12,4,26,14,1.130,2021-05-16,False
32561,"2021-05-18 - Flames 2, Canucks 4",Vancouver,30,22,3,1.32,17,6,5,11,9,17,8,1.103,2021-05-18,True
32560,"2021-05-18 - Flames 2, Canucks 4",Calgary,35,30,1,2.36,25,10,9,15,11,15,10,0.897,2021-05-18,False
32563,"2021-05-19 - Canucks 2, Flames 6",Vancouver,29,18,1,1.65,18,6,4,12,5,15,8,0.816,2021-05-19,True


In [ ]:
# Drop the 'Game' column as we don't need it anymore, and combine everything into a single row
combined_df3.drop(columns=['Game', 'away_row'], inplace = True)

# It is not always visitors row, then home row. Sometimes home row is first.
# So, we have to decide to shift-1 = visitors/home, or shift+1 = home/visitors

# Visitors Row
# Home Row
combined_df3['Home_Team'] = combined_df3.Team.shift(-1)
combined_df3['Home_FF'] = combined_df3.FF.shift(-1)
combined_df3['Home_SF'] = combined_df3.SF.shift(-1)
combined_df3['Home_GF'] = combined_df3.GF.shift(-1)
combined_df3['Home_xGF'] = combined_df3.xGF.shift(-1)
combined_df3['Home_SCF'] = combined_df3.SCF.shift(-1)
combined_df3['Home_HDCF'] = combined_df3.HDCF.shift(-1)
combined_df3['Home_HDSF'] = combined_df3.HDSF.shift(-1)
combined_df3['Home_MDCF'] = combined_df3.MDCF.shift(-1)
combined_df3['Home_MDSF'] = combined_df3.MDSF.shift(-1)
combined_df3['Home_LDCF'] = combined_df3.LDCF.shift(-1)
combined_df3['Home_LDSF'] = combined_df3.LDSF.shift(-1)
combined_df3['Home_PDO'] = combined_df3.PDO.shift(-1)

combined_df3


,Team,FF,SF,GF,xGF,SCF,HDCF,HDSF,MDCF,MDSF,...,Home_GF,Home_xGF,Home_SCF,Home_HDCF,Home_HDSF,Home_MDCF,Home_MDSF,Home_LDCF,Home_LDSF,Home_PDO
0,Anaheim,18,14,0,1.28,15,6,4,9,5,...,0.0,0.34,7.0,0.0,0.0,7.0,5.0,2.0,0.0,1.000
1,LosAngeles,9,6,0,0.34,7,0,0,7,5,...,2.0,1.53,19.0,9.0,8.0,10.0,5.0,10.0,5.0,1.055
2,Anaheim,26,19,2,1.53,19,9,8,10,5,...,1.0,1.20,16.0,3.0,3.0,13.0,7.0,13.0,10.0,0.945
3,LosAngeles,26,20,1,1.20,16,3,3,13,7,...,1.0,2.15,26.0,12.0,7.0,14.0,5.0,15.0,10.0,1.045
4,Carolina,32,22,1,2.15,26,12,7,14,5,...,0.0,2.13,27.0,11.0,9.0,16.0,8.0,19.0,6.0,0.955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32558,Calgary,34,23,5,1.73,19,7,5,12,4,...,3.0,1.32,17.0,6.0,5.0,11.0,9.0,17.0,8.0,1.103
32561,Vancouver,30,22,3,1.32,17,6,5,11,9,...,1.0,2.36,25.0,10.0,9.0,15.0,11.0,15.0,10.0,0.897
32560,Calgary,35,30,1,2.36,25,10,9,15,11,...,1.0,1.65,18.0,6.0,4.0,12.0,5.0,15.0,8.0,0.816
32563,Vancouver,29,18,1,1.65,18,6,4,12,5,...,6.0,3.15,30.0,19.0,13.0,11.0,6.0,13.0,6.0,1.184


In [ ]:
# Drop odd rows as they're no longer needed and inaccurate

combined_df4 = combined_df3.iloc[::2, :].copy()

# Rename away columns
home_columns = {'Team': 'Away_Team', 'FF': 'Away_FF', 'SF': 'Away_SF', 'GF': 'Away_GF', 'xGF': 'Away_xGF', 'SCF': 'Away_SCF',\
    'HDCF': 'Away_HDCF', 'HDSF': 'Away_HDSF', 'MDCF': 'Away_MDCF', 'MDSF': 'Away_MDSF',\
    'LDCF': 'Away_LDCF', 'LDSF': 'Away_LDSF', 'PDO': 'Away_PDO'}

combined_df4.rename(columns=home_columns, inplace=True)

combined_df4

,Away_Team,Away_FF,Away_SF,Away_GF,Away_xGF,Away_SCF,Away_HDCF,Away_HDSF,Away_MDCF,Away_MDSF,...,Home_GF,Home_xGF,Home_SCF,Home_HDCF,Home_HDSF,Home_MDCF,Home_MDSF,Home_LDCF,Home_LDSF,Home_PDO
0,Anaheim,18,14,0,1.28,15,6,4,9,5,...,0.0,0.34,7.0,0.0,0.0,7.0,5.0,2.0,0.0,1.000
2,Anaheim,26,19,2,1.53,19,9,8,10,5,...,1.0,1.20,16.0,3.0,3.0,13.0,7.0,13.0,10.0,0.945
4,Carolina,32,22,1,2.15,26,12,7,14,5,...,0.0,2.13,27.0,11.0,9.0,16.0,8.0,19.0,6.0,0.955
6,Anaheim,12,7,1,0.78,9,3,2,6,3,...,0.0,1.92,22.0,8.0,6.0,14.0,7.0,12.0,8.0,0.857
8,Ottawa,31,21,3,1.60,22,8,4,14,8,...,3.0,1.77,29.0,9.0,4.0,20.0,9.0,13.0,7.0,1.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,Winnipeg,31,20,3,1.77,18,6,6,12,6,...,2.0,2.53,22.0,9.0,4.0,13.0,8.0,26.0,16.0,0.919
32557,Vancouver,40,34,3,1.55,22,4,3,18,11,...,1.0,1.74,19.0,9.0,7.0,10.0,5.0,17.0,10.0,0.955
32559,Vancouver,25,23,2,1.33,17,3,3,14,11,...,5.0,1.73,19.0,7.0,5.0,12.0,4.0,26.0,14.0,1.130
32561,Vancouver,30,22,3,1.32,17,6,5,11,9,...,1.0,2.36,25.0,10.0,9.0,15.0,11.0,15.0,10.0,0.897


In [ ]:
# Export it to a CSV for use in next step
combined_df4.to_csv('nst_2008_2021.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f0efbe77-01fa-4860-b5ee-e7eac30d44e8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>